# Documentation assistent

This notebook demonstrates a documentation assistent:
1. Video-to-protocol conversion using Vertex AI
2. With knowledge from documents and pictures that are loaded into cache

In [1]:
# %pip install google-cloud-storage
# %pip install --upgrade --user --quiet google-cloud-aiplatform

In [2]:
from __future__ import annotations

# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

import configparser
import os
import sys
from pathlib import Path

from IPython.display import Markdown

path_to_append = Path(Path.cwd()).parent / "proteomics_specialist"
sys.path.append(str(path_to_append))
import video_to_protocol

config = configparser.ConfigParser()
config.read("../secrets.ini")

['../secrets.ini']

In [3]:
import configparser

import vertexai

config = configparser.ConfigParser()
config.read("../secrets.ini")

PROJECT_ID = config["DEFAULT"]["PROJECT_ID"]
vertexai.init(project=PROJECT_ID, location="europe-west9")  # europe-west9 is Paris

In [4]:
from google.cloud import storage

os.environ["GOOGLE_CLOUD_PROJECT"] = config["DEFAULT"]["PROJECT_ID"]

# Initialize Cloud Storage client
storage_client = storage.Client()
bucket_name = "mannlab_videos"
bucket = storage_client.bucket(bucket_name)

In [5]:
from vertexai.generative_models import Part
from vertexai.preview.generative_models import GenerativeModel

MODEL_ID = "gemini-1.5-pro-001"

# Following: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/context-caching/intro_context_caching_vertex_ai_sdk.ipynb

In [6]:
# Upload knowledge files to Google Cloud Storage
folder_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/knowledge_base"
subfolder_in_bucket = "knowledge"

knowledge_uris = []
for filename in os.listdir(folder_path):
    if filename.lower().endswith(
        (".jpg", ".jpeg", ".gif", ".bmp", ".tiff", ".tif", ".pdf")
    ):
        path = Path(folder_path) / filename
        try:
            file_uri = video_to_protocol.upload_video_to_gcs(
                path, bucket, subfolder_in_bucket
            )
            knowledge_uris.append(file_uri)
        except OSError as e:
            print(f"Error processing {filename}: {e}")

In [7]:
# Create cache with Vertex AI
cached_content = video_to_protocol.create_cached_content(
    knowledge_uris, model_id=MODEL_ID
)

Total files processed: 40
  PDF: 40
Cached content created successfully!


In [8]:
# cached_content.delete()

In [9]:
print(cached_content.name)
# print(cached_content.resource_name)
# print(cached_content.model_name)
print(cached_content.create_time)
print(cached_content.expire_time)

9195541198532509696
2025-03-24 11:54:45.408869+00:00
2025-03-24 12:54:45.402552+00:00


In [10]:
model = GenerativeModel.from_cached_content(cached_content=cached_content)

In [11]:
video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/DisconnectColumn_protocolCorrect.MP4"
video_uri_example_1 = video_to_protocol.upload_video_to_gcs(video_path, bucket)

path = "/Users/patriciaskowronek/Documents/proteomics_specialist/data/DisconnectColumn_protocolCorrect.md"
protocol_uri_example_1 = video_to_protocol.upload_video_to_gcs(path, bucket)

video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/PlaceEvotips_protocolCorrect.MP4"
video_uri_example_2 = video_to_protocol.upload_video_to_gcs(video_path, bucket)

path = "/Users/patriciaskowronek/Documents/proteomics_specialist/data/PlaceEvotips_protocolCorrect.md"
protocol_uri_example_2 = video_to_protocol.upload_video_to_gcs(path, bucket)

video_path = "/Users/patriciaskowronek/Documents/documentation_agent_few_shot_examples/benchmark_dataset/ConnectingColumnSampleLine_protocolCorrect.MP4"
video_uri_input = video_to_protocol.upload_video_to_gcs(video_path, bucket)

## Agent 1: The observer

In [12]:
system_prompt_1 = """
You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry.
Your task is to analyze the provided video and to write a clear, concise, unambiguous documentation.
Take deep breath and think step-by-step with me. Answer direct.
"""

In [13]:
example_documentation_1 = """
1. Describe what you can hear with timestamps:
    - 0:00 - 0:08 "Okay, I want to disconnect um the ion optics column which is here"
    - 0:09 - 0:17 "from the sample line of the Evosep. So for this we first have to go to the tims control software and check if it is"
    - 0:18 - 0:24 "in standby mode. At the moment you can see here it's in operating mode so we just click on the on-off signal"
    - 0:25 - 0:29 "and it's here in standby. Good."
    - 0:30 - 0.39 "Let's move back to the instrument. As a next step we have to open or I open the lid of the column oven which is also called"
    - 0:40 - 0:48 "column toaster. Then there is this screw which is um to that's the securing screw which does"
    - 0:49 - 0:59 "ground the ion optics column so that we have a um stable spray so I just lift uh lifted it up and"
    - 1:00 - 1:11 "yeah loosen it so that is not anymore above the connection. Then I attach um a nano viper adapter here to the sample line"
    - 1:12 - 1:19 "So it's just like pushing it on top of it. I find it easier to hold like the column fitting with pillars"
    - 1:20 - 1:31 "and then to open the column counterclockwise. So I find this a little bit easier for handling. I unscrew here the nano connector"
    - 1:32 - 1:43 "from the column fitting um and then so that the sample line at the Evosep and the ion optics column are detached. And now I put position here the sample line"
    - 1:44 - 1:53 "on top of the bumper of the Evosep just to have it out of the way and very can um yeah where it can"
    - 1:54 - 2:05 "then uh the idle flow can flow. So now ion optics column and sample line are disconnected."

2. Describe what you can see with timestamps:
    - 0:00 - 0:07 The video starts with a view of the UltraSource of a timsTOF Ultra instrument. The camera pans down since the researcher shows a box containing an IonOpticks column.
    - 0:08 - 0:12 The researcher points on the sample line connected with the Evosep One LC system.
    - 0:13 - 0:21 The researcher walks to the computer monitor displaying the timsControl software.
    - 0:22 - 0:22 The researcher pointes to the "operating" software status.
    - 0:23 - 0:27 The researcher clicks on the "on/off Instrument" button, which changes the software status to "Standing by".
    - 0:28 - 0:32 The researcher walks back to the timsTof Ultra.
    - 0:33 - 0:40 The researcher opens the lid of the column oven, which is also called the column toaster.
    - 0:41 - 1.03 The researcher points to a securing screw that grounds the ion optics column. They lift the screw and move it away so that it cannot make a connection with the column fitting anymore.
    - 1.04 - 1:13 The researcher grabs a NanoViper adapter and attaches it to the sample line by gently pushing it onto the NanoViper connector.
    - 1:14 - 1:20 The researcher grabs pliers and holds the column fitting with them.
    - 1:21 - 1:32 The researcher uses their other hand to unscrew the nano connector from the column fitting counter-clockwise.
    - 1:33 - 1:40 The researcher explains and shows the detached sample line connector and column fitting.
    - 1:41 - 1:43 The researcher places the sample line on top of the bumper of the Evosep One LC system.
    - 1:44 - 1:47 The researcher places the pliers back on the Evosep carrier.
    - 1:48 - 2:05 The researcher confirms that the ion optics column and sample line are now disconnected.

3. Describe the used equipment:
    - timsTOF Ultra instrument: Black and silver, with a round, black source housing on the right side. The instrument is labeled "timsTOF SCP".
    - IonOpticks column: Small, cylindrical column packaged in a blue and white box.
    - Evosep One LC system: Orange and white, with a transparent bumper. The system is labeled "EVOSEP ONE".
    - timsControl software: Displayed on a computer monitor. The software interface shows various tabs, including "Instrument", "Method", "Acquisition", "Home", "Maintenance", and "Monitoring".
    - Column oven: Small, rectangular oven attached to the timsTOF Ultra instrument. The oven has a black lid and is also called "Column Toaster".
    - Sample line: Thin, blue tubing connected to the IonOpticks column.
    - NanoViper adapter: Small, black fitting attached to the sample line.
    - Pliers: Red and yellow, used to hold the column fitting of the IonPticks column.
"""

example_documentation_2 = """
1. Describe what you can hear with timestamps:

0:00 - 0:07 "I want to place six Evotips from position A1 to A6."
0:08 - 0:21 "So first I need an Evotip box and check that there's some solvent inside, so it should be like 1 centimeter, which is fine."
0:21 - 0:31 "Then I place this Evotip box on top here of the rack and make sure that it's securely seated within the rack."
0:32 - 0:39 "I do the same with an empty box where I will fill in blanks."
0:40 - 0:46 "Next, I will add some Evotips. For this I have here some prepared."
0:47 - 1:11 "I look at them visually if they look fine. They have like some solvent on top, on the bottom, and like the um SPE material, this white material, it has like a pale color, so I believe they are fine. So I check all of them by hand."
1:12 - 1:20 "Then I do the same with the blanks."
1:21 - 1:30 "Here like they are completely dry, so these are just unused tips, not cleaned, nothing, so whatever. Uh, which you just did here."
1:31 - 1:44 "Good. And now as a last step, um I remember or note down that they are from A1 to A6 on the rack one and here on rack three. They are also from A1 to A6."

2. Describe what you can see with timestamps:

0:00 - 0:08 The video starts with a view of the Evosep One instrument. The researcher is wearing black gloves.
0:09 - 0:22 The researcher picks up a yellow Evotip box and checks the solvent level.
0:23 - 0:31 They then place the box in position S1 on the Evosep One instrument and ensure that it is well seated.
0:32 - 0:40 They repeat this with an empty Evotip box, placing it in position S3.
0:41 - 0:56 The researcher picks up a box of Evotips and visually inspects them, checking for solvent and the color of the SPE material.
0:57 - 1:13 They then place the Evotips in S1 from A1 to A6 on the Evosep One instrument.
1:14 - 1:31 The researcher picks up a box of empty Evotips and places tem in S3 from A1 to A6.
1:32 - 1:03 The researcher points to the Evotip boxes and verbally confirms their positions.

3. Describe the used equipment:

Evosep One instrument: White and orange, with a transparent bumper and a sample tray with six positions.
Evotip boxes: Transparent box. Each box holds 96 Evotips.
Evotips: transparent pipette tips with a white material at the bottom.
"""

In [14]:
inputs = [
    system_prompt_1,
    "EXAMPLE SECTION: The following is an EXAMPLE of the video, documentation.",
    "Example Video 1:",
    Part.from_uri(video_uri_example_1, mime_type="video/mp4"),
    "Example documentation 1:",
    example_documentation_1,
    "Example Video 2:",
    Part.from_uri(video_uri_example_2, mime_type="video/mp4"),
    "Example documentation 2:",
    example_documentation_2,
    "END OF EXAMPLE SECTION",
    "YOUR TASK SECTION:",
    "Analyze the following NEW video and create a documentation for it. Video:",
    Part.from_uri(video_uri_input, mime_type="video/mp4"),
    "Documentation:",
]

response = model.generate_content(inputs, generation_config={"temperature": 0})
observation = response.text
print(response.usage_metadata)
Markdown(observation)

prompt_token_count: 374808
candidates_token_count: 1346
total_token_count: 376154
cached_content_token_count: 261613
prompt_tokens_details {
  modality: TEXT
  token_count: 2046
}
prompt_tokens_details {
  modality: VIDEO
  token_count: 111150
}
prompt_tokens_details {
  modality: DOCUMENT
  token_count: 261612
}
candidates_tokens_details {
  modality: TEXT
  token_count: 1346
}



1. Describe what you can hear with timestamps:

0:00 - 0:08 "I want to connect here from the column the uh fitting with the sample line connector."
0:08 - 0:12 "First let's check the software starters in tims control."
0:16 - 0:25 "It's currently in standby. If it would have been in operate mode like now, we would have had to click here and switch it to standby."
0:25 - 0:35 "For ion optics columns it's important that they're not left for an extended period of time in operate mode or in standby as soon as there is some flow on the column."
0:40 - 0:48 "So the protocol assumes that the ion optics column is already inserted into the UltraSource which is the case. Then let's go to the sample line. Here we connect um the adapter for the nano connector."
0:57 - 1:15 "Just push it on it. Then we check if there's some liquid on top. If I just snip it away. Then I hold column fitting with pillars and I hand tightened the sample line into the column connector."
1:16 - 1:24 "Here it's important to have it finger tight but not over tighten it. Then I usually already removed the adapter."
1:25 - 1:34 "As a next step um you would have to check if um here the column oven is at the right position. Here is like a um"
1:35 - 1:42 "screw which can be loosened to move the oven and it should be as close as possible to the source."
1:42 - 1:54 "Then here is a screw which should be on top of the connection. It makes sure that um the column is grounded. If it's a longer column, it also could be grounded with this screw or with that screw."
1:55 - 2:02 "Okay. That's all fine. So we will close the column oven."
2:03 - 2:14 "And then check that it's at 50 degrees which is indicated by three lights. It's blinking so this just means that it has to heat up to have the full temperature. Okay."
2:15 - 2:18 "Next we go to tims control."
2:21 - 2:24 "Switching to operate mode."
2:25 - 2:29 "And then go to high star and check if idle flow is on. It is."
2:30 - 2:36 "Otherwise we could have right-clicked and say here idle flow run, but it's already on. No reason to do this by hand."
2:37 - 2:43 "So it's operating and we have signal. Wonderful."

2. Describe what you can see with timestamps:

0:00 - 0:08 The video starts with a view of the UltraSource of a timsTOF SCP instrument. The researcher is wearing black gloves.
0:09 - 0:15 The researcher walks to the computer monitor displaying the timsControl software.
0:16 - 0:24 The researcher points to the "Standby" software status and clicks on the "on/off Instrument" button, which changes the software status to "Operating".
0:25 - 0:39 The researcher walks back to the timsTOF SCP.
0:40 - 0:57 The researcher picks up a NanoViper adapter and attaches it to the sample line by gently pushing it onto the NanoViper connector.
0:58 - 1:15 The researcher grabs pliers and holds the column fitting with them. They use their other hand to screw the nano connector from the sample line into the column fitting clockwise.
1:16 - 1:24 The researcher removes the NanoViper adapter.
1:25 - 1:41 The researcher points to a screw that can be loosened to move the column oven closer to the source.
1:42 - 1:54 The researcher points to a screw that grounds the column. They explain that for longer columns, the column can also be grounded with a screw on the column oven.
1:55 - 2:02 The researcher closes the lid of the column oven.
2:03 - 2:14 The researcher points to the column oven and explains that the three blinking lights indicate that it is heating up to 50 degrees.
2:15 - 2:24 The researcher walks to the computer monitor displaying the timsControl software and switches the instrument to "Operate" mode.
2:25 - 2:36 The researcher walks to the computer monitor displaying the HyStar software and checks if the idle flow is on.
2:37 - 2:43 The researcher walks back to the timsControl software and confirms that the instrument is operating and that there is signal.

3. Describe the used equipment:

- timsTOF SCP instrument: Black and silver, with a round, black source housing on the right side. The instrument is labeled "timsTOF SCP".
- IonOpticks column: Small, cylindrical column.
- Evosep One LC system: Orange and white, with a transparent bumper. The system is labeled "EVOSEP ONE".
- timsControl software: Displayed on a computer monitor. The software interface shows various tabs, including "Instrument", "Method", "Acquisition", "Home", "Maintenance", and "Monitoring".
- Column oven: Small, rectangular oven attached to the timsTOF SCP instrument. The oven has a black lid and is also called "Column Toaster".
- Sample line: Thin, blue tubing connected to the IonOpticks column.
- NanoViper adapter: Small, black fitting attached to the sample line.
- Pliers: Red and yellow, used to hold the column fitting of the IonPticks column.
- HyStar software: Displayed on a computer monitor. The software interface shows a sample table and an instrument status window.

## Agent 2: Documentation to protocol

In [15]:
system_prompt_agent2 = """
You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry.
Your task is to convert the given documentation into a clear, concise, unambiguous protocol.
Take deep breath and think step-by-step with me. Answer direct.
"""

In [16]:
inputs = [
    system_prompt_agent2,
    "EXAMPLE SECTION: The following is an EXAMPLE of video documentation and the converted protocol.",
    "Example documentation 1:",
    example_documentation_1,
    "Example protocol 1:",
    Part.from_uri(
        protocol_uri_example_1,
        mime_type="text/md",
    ),
    "Example documentation 2:",
    example_documentation_2,
    "Example protocol 2:",
    Part.from_uri(
        protocol_uri_example_2,
        mime_type="text/md",
    ),
    "END OF EXAMPLE SECTION",
    "FORMATTING INSTRUCTIONS:",
    """
    It is of highest importance for you to exactly follow the format of the example protocol with these sections:
    1. Title (format: **# Title**)
    2. Abstract (format: **## Abstract** followed by a paragraph)
    3. Materials section with Equipment and Reagents subsections
        (format: **## Materials**
                **### Equipment**
                - **Item 1**
                - **Item 2**)
    4. Procedure with estimated timing
        (format: **## Procedure**
                *Estimated timing: X minutes*
                1. Step one
                2. Step two)
    5. Expected Results section (format: **## Expected Results**)
    6. Figures section (format: **## Figures**)
    7. References section (format: **## References**)
    """,
    "YOUR TASK SECTION:",
    "Convert the documentation to a protocol. Documentation:",
    observation,
    "Protocol:",
]

response = model.generate_content(
    inputs,
    generation_config={"temperature": 0},  # try 0.9
)
protocol = response.text
print(response.usage_metadata)
Markdown(protocol)

# Make sure you do not include any steps that are not shown in the video
# include timestap command in model call

prompt_token_count: 266464
candidates_token_count: 1453
total_token_count: 267917
cached_content_token_count: 261613
prompt_tokens_details {
  modality: TEXT
  token_count: 4852
}
prompt_tokens_details {
  modality: DOCUMENT
  token_count: 261612
}
candidates_tokens_details {
  modality: TEXT
  token_count: 1453
}



==Start of OCR for page 1==
# Connecting an IonOpticks column to the timsTOF SCP
## Abstract
This protocol describes how to connect an IonOpticks column to a timsTOF SCP instrument using an Evosep One LC system. It includes steps for disconnecting the column, connecting the new column, and ensuring proper grounding and temperature control.
## Materials
### Equipment
- timsTOF SCP instrument
- IonOpticks column
- Evosep One LC system
- Column oven (Column Toaster)
- Sample line
- NanoViper adapter
- Pliers
### Reagents
- Acetonitrile (HPLC LC-MS grade)
## Procedure
*Estimated timing: 10 minutes*
1. **Switch the timsTOF SCP to standby mode.** In the timsControl software, click on the "on/off Instrument" button to change the software status from "Operating" to "Standby".
2. **Disconnect the old column.**
    a. Open the lid of the column oven (Column Toaster).
    b. Loosen the securing screw that grounds the ion optics column.
    c. Attach a NanoViper adapter to the sample line by gently pushing it onto the NanoViper connector.
    d. Hold the column fitting with pliers and unscrew the nano connector from the column fitting counter-clockwise.
    e. Place the sample line on top of the bumper of the Evosep One LC system.
3. **Connect the new IonOpticks column.**
    a. Attach a NanoViper adapter to the sample line.
    b. Screw the nano connector from the sample line into the column fitting clockwise.
    c. Remove the NanoViper adapter.
    d. Ensure that the column oven is as close as possible to the source.
    e. Ensure that the column is grounded. For shorter columns, use the screw on the source housing. For longer columns, use the screw on the column oven.
    f. Close the lid of the column oven.
    g. Ensure that the column oven is set to 50 degrees.
4. **Switch the timsTOF SCP to operate mode.** In the timsControl software, click on the "on/off Instrument" button to change the software status from "Standby" to "Operating".
5. **Check if the idle flow is on.** In the HyStar software, check if the idle flow is active. If not, right-click on the Evosep logo, select "idle flow", and then click "Run".
## Expected Results
After following this protocol, the IonOpticks column should be properly connected to the timsTOF SCP instrument and ready for use. The instrument should be in operate mode, the idle flow should be active, and the column oven should be at the correct temperature.
## Figures
- Extended Data Fig. 1: TimsControl settings
- Extended Data Fig. 2: Navigating in HyStar
- Supplementary Method Figure 1: UltraSource
- Supplementary Method Figure 4: Connection of a sample line of a LC system with a separation column and emitter.
## References
- Meier, F. et al. Online parallel accumulation-serial fragmentation (PASEF) with a novel trapped ion mobility mass spectrometer. Mol. Cell. Proteom. 17, 2534-2545 (2018).
- Skowronek, P. et al. Rapid and In-Depth Coverage of the (Phospho-)Proteome With Deep Libraries and Optimal Window Design for dia-PASEF. Mol. Cell. Proteomics 21, 100279 (2022).
# Loading Evotips on the Evosep One
## Abstract
This protocol describes how to load Evotips on the Evosep One instrument for proteomics experiments. It includes steps for rinsing, conditioning, equilibrating, loading, washing, and preserving the Evotips.
## Materials
### Equipment
- Evosep One instrument
- Evotip boxes
- Evotips
- Single-well plate
- Rack holder with bars
- Table centrifuge
- Multichannel pipettes suitable for 10 µl, 20 µl, 100 µl, and 150 µl volumes
### Reagents
- Solvent A: Mass Spec Grade Water with 0.1% Formic Acid
- Solvent B: Mass Spec Grade Acetonitrile with 0.1% Formic Acid
- 1-Propanol (alternatively 2-Propanol)
## Procedure
*Estimated timing: 20 minutes*
1. **Rinse.**
    a. Transfer 20 µl of Solvent B to all Evotips.
    b. Place the Evotips in the centrifuge with appropriate counter balance and centrifuge at 800 g for 60 seconds.
    c. Empty the box for liquid after centrifugation.
2. **Condition.**
    a. Place the Evotip adapter rack on top of the single-well plate with 1-Propanol.
    b. Soak for a minimum of 10 seconds and then visually inspect that Evotips are pale white.
3. **Equilibrate.**
    a. With the Evotips still in the adapter rack, transfer 20 µl of Solvent A to each Evotip.
    b. Place the Evotips in the centrifuge with appropriate counter balance and centrifuge at 800 g for 60 seconds.
    c. Empty the box for liquid after centrifugation.
4. **Load sample.**
    a. Place the Evotips in the original tray and transfer 20 µl of the sample to each Evotip.
    b. Place the Evotips in the centrifuge with counter balance and centrifuge at 800 g for 60 seconds.
    c. Empty the box for liquid after centrifugation.
5. **Wash.**
    a. Transfer 20 µl of Solvent A to each Evotip.
    b. Place the Evotips in the centrifuge with appropriate counter balance and centrifuge at 800 g for 60 seconds.
    c. Empty the box for liquid after centrifugation.
6. **Preservation.**
    a. Transfer 100 µl of Solvent A to each Evotip.
    b. Place the Evotips in the centrifuge with appropriate counter balance and centrifuge at 800 g for 10 seconds to keep Evotips wet.
7. **Storage.** Add enough Solvent A to the bottom of the tray so the bottom of the Evotips are submerged and store at 4°C with lid on until analysis.
## Expected Results
After following this protocol, the Evotips should be properly loaded with sample and ready for use on the Evosep One instrument.
## Figures
- None
## References
- None


In [17]:
protocol

'==Start of OCR for page 1==\n# Connecting an IonOpticks column to the timsTOF SCP\n## Abstract\nThis protocol describes how to connect an IonOpticks column to a timsTOF SCP instrument using an Evosep One LC system. It includes steps for disconnecting the column, connecting the new column, and ensuring proper grounding and temperature control.\n## Materials\n### Equipment\n- timsTOF SCP instrument\n- IonOpticks column\n- Evosep One LC system\n- Column oven (Column Toaster)\n- Sample line\n- NanoViper adapter\n- Pliers\n### Reagents\n- Acetonitrile (HPLC LC-MS grade)\n## Procedure\n*Estimated timing: 10 minutes*\n1. **Switch the timsTOF SCP to standby mode.** In the timsControl software, click on the "on/off Instrument" button to change the software status from "Operating" to "Standby".\n2. **Disconnect the old column.**\n    a. Open the lid of the column oven (Column Toaster).\n    b. Loosen the securing screw that grounds the ion optics column.\n    c. Attach a NanoViper adapter to th

## Agent 3: Check protocol

In [18]:
system_prompt_agent3 = system_prompt_verification = """
You are Professor Matthias Mann, a pioneering scientist in proteomics and mass spectrometry.
Your scientific reputation depends on absolute accuracy in this verification task.

TASK: Critically analyze a scientific protocol against a video to identify steps that are NOT actually shown.

FIRST TASK - VIDEO OBSERVATION:
1. Create a timestamped log of EVERYTHING you observe in the video, second by second
2. Be extremely literal - only include actions you can physically see happening
3. Do NOT infer or assume actions that might have happened between visible segments

SECOND TASK - PROTOCOL VERIFICATION:
For each numbered step in the protocol:
1. Search your observation log for DIRECT evidence of this exact action
2. If you cannot find a timestamp where this SPECIFIC action is clearly visible, mark it as "NO - NOT SHOWN"
3. Do NOT give a step credit for being "implied" or "probably happened" - it must be DIRECTLY OBSERVABLE

Format your response for each step exactly as:

Step X: [YES/NO]
Timestamp: [Exact time or "NOT SHOWN"]
Evidence: [Describe EXACTLY what you see at this timestamp that matches this step]
Terminology: [Correct/Incorrect]

CRITICAL INSTRUCTION: After analyzing all steps, list "STEPS NOT SHOWN IN VIDEO: X, Y, Z" and then create a corrected protocol that completely removes these steps.

VERIFICATION STANDARDS:
- A step is only "shown" if you can see the COMPLETE action described
- Partial evidence is not sufficient - mark as "NOT SHOWN"
- When in doubt, default to "NOT SHOWN"
- Assume this verification is being audited by skeptical scientists
"""

In [19]:
inputs = [
    system_prompt_agent3,
    "Video:",
    Part.from_uri(video_uri_input, mime_type="video/mp4"),
    "Your task: Analyze following protocol:",
    protocol,
]

response = model.generate_content(inputs, generation_config={"temperature": 0})
improved = response.text
print(response.usage_metadata)
Markdown(improved)

prompt_token_count: 309878
candidates_token_count: 2986
total_token_count: 312864
cached_content_token_count: 261613
prompt_tokens_details {
  modality: TEXT
  token_count: 1811
}
prompt_tokens_details {
  modality: DOCUMENT
  token_count: 261612
}
prompt_tokens_details {
  modality: VIDEO
  token_count: 46455
}
candidates_tokens_details {
  modality: TEXT
  token_count: 2986
}



## Protocol Verification Against Video

**VIDEO OBSERVATION LOG:**

0:00 - Gloved hands holding a column fitting and a NanoViper adapter.
0:02 - Hands move towards the timsTOF SCP instrument.
0:03 - Hands hold the column fitting and a sample line connector.
0:06 - Hands connect the sample line connector to the column fitting using pliers.
0:10 - Camera pans to show the Evosep One LC system and the timsTOF SCP instrument.
0:16 - Camera focuses on the timsControl software on a computer screen.
0:22 - Hands connect the sample line connector to the column fitting using pliers.
0:23 - Hands remove the NanoViper adapter.
0:30 - Hands point to a screw on the column oven.
0:35 - Hands point to a screw on the source housing.
0:37 - Hands loosen a screw on the column oven.
0:41 - Hands tighten a screw on top of the column connection.
0:47 - Hands point to the column oven and the source.
0:56 - Hands point to the column oven.
1:04 - Hands remove the adapter.
1:05 - Hands connect the sample line connector to the column fitting using pliers.
1:21 - Hands remove the adapter.
1:26 - Camera pans to show the timsControl software on a computer screen.
1:31 - Hands point to the "Operate" button on the timsControl software.
1:35 - Camera pans to show the Evosep One LC system.
1:40 - Camera focuses on the HyStar software on a computer screen.
1:50 - Hands point to the Evosep One LC system.
2:00 - Hands close the lid of the column oven.
2:06 - Hands point to the column oven.
2:16 - Camera pans to show the timsControl software on a computer screen.
2:21 - Hands point to the "Operate" button on the timsControl software.
2:25 - Camera pans to show the HyStar software on a computer screen.
2:30 - Hands point to the Evosep One LC system on the HyStar software.
2:37 - Hands point to the "Operate" button on the timsControl software.

**PROTOCOL VERIFICATION:**

**Connecting an IonOpticks column to the timsTOF SCP**

**Step 1:** [YES]
Timestamp: 0:16
Evidence: The timsControl software is visible on the computer screen, and the user states that the instrument is currently in standby mode.
Terminology: Correct

**Step 2a:** [YES]
Timestamp: 0:02
Evidence: The gloved hands move towards the timsTOF SCP instrument, implying that they are going to open the column oven.
Terminology: Correct

**Step 2b:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: The loosening of the securing screw that grounds the ion optics column is not shown in the video.
Terminology: Correct

**Step 2c:** [YES]
Timestamp: 0:00
Evidence: The gloved hands are holding a NanoViper adapter at the beginning of the video.
Terminology: Correct

**Step 2d:** [YES]
Timestamp: 0:06
Evidence: The gloved hands are holding the column fitting with pliers and unscrewing the nano connector from the column fitting.
Terminology: Correct

**Step 2e:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Placing the sample line on top of the bumper of the Evosep One LC system is not shown in the video.
Terminology: Correct

**Step 3a:** [YES]
Timestamp: 0:00
Evidence: The gloved hands are holding a NanoViper adapter at the beginning of the video.
Terminology: Correct

**Step 3b:** [YES]
Timestamp: 1:05
Evidence: The gloved hands are holding the column fitting with pliers and screwing the nano connector from the sample line into the column fitting.
Terminology: Correct

**Step 3c:** [YES]
Timestamp: 1:21
Evidence: The gloved hands remove the NanoViper adapter.
Terminology: Correct

**Step 3d:** [YES]
Timestamp: 0:37
Evidence: The user states that the column oven should be as close as possible to the source and loosens a screw on the column oven to adjust its position.
Terminology: Correct

**Step 3e:** [YES]
Timestamp: 0:30, 0:35
Evidence: The user states that the column should be grounded and points to the screws on the column oven and the source housing that can be used for grounding.
Terminology: Correct

**Step 3f:** [YES]
Timestamp: 2:00
Evidence: The gloved hands close the lid of the column oven.
Terminology: Correct

**Step 3g:** [YES]
Timestamp: 2:06
Evidence: The user states that the column oven should be set to 50 degrees and points to the column oven, which has three lights blinking, indicating that it is heating up.
Terminology: Correct

**Step 4:** [YES]
Timestamp: 2:21
Evidence: The user clicks on the "Operate" button in the timsControl software, changing the software status from "Standby" to "Operating".
Terminology: Correct

**Step 5:** [YES]
Timestamp: 2:25
Evidence: The HyStar software is visible on the computer screen, and the user states that the idle flow is already on.
Terminology: Correct

**Loading Evotips on the Evosep One**

**Step 1a:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Transferring 20 µl of Solvent B to all Evotips is not shown in the video.
Terminology: Correct

**Step 1b:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Placing the Evotips in the centrifuge and centrifuging them is not shown in the video.
Terminology: Correct

**Step 1c:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Emptying the box for liquid after centrifugation is not shown in the video.
Terminology: Correct

**Step 2a:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Placing the Evotip adapter rack on top of the single-well plate with 1-Propanol is not shown in the video.
Terminology: Correct

**Step 2b:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Soaking the Evotips in 1-Propanol and visually inspecting them is not shown in the video.
Terminology: Correct

**Step 3a:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Transferring 20 µl of Solvent A to each Evotip is not shown in the video.
Terminology: Correct

**Step 3b:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Placing the Evotips in the centrifuge and centrifuging them is not shown in the video.
Terminology: Correct

**Step 3c:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Emptying the box for liquid after centrifugation is not shown in the video.
Terminology: Correct

**Step 4a:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Placing the Evotips in the original tray and transferring 20 µl of the sample to each Evotip is not shown in the video.
Terminology: Correct

**Step 4b:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Placing the Evotips in the centrifuge and centrifuging them is not shown in the video.
Terminology: Correct

**Step 4c:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Emptying the box for liquid after centrifugation is not shown in the video.
Terminology: Correct

**Step 5a:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Transferring 20 µl of Solvent A to each Evotip is not shown in the video.
Terminology: Correct

**Step 5b:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Placing the Evotips in the centrifuge and centrifuging them is not shown in the video.
Terminology: Correct

**Step 5c:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Emptying the box for liquid after centrifugation is not shown in the video.
Terminology: Correct

**Step 6a:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Transferring 100 µl of Solvent A to each Evotip is not shown in the video.
Terminology: Correct

**Step 6b:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Placing the Evotips in the centrifuge and centrifuging them is not shown in the video.
Terminology: Correct

**Step 7:** [NO - NOT SHOWN]
Timestamp: NOT SHOWN
Evidence: Adding Solvent A to the bottom of the tray and storing the Evotips at 4°C is not shown in the video.
Terminology: Correct

**STEPS NOT SHOWN IN VIDEO:** 2b, 2e, 3b, 3c, 4a, 4b, 4c, 5a, 5b, 5c, 6a, 6b, 7

**CORRECTED PROTOCOL:**

# Connecting an IonOpticks column to the timsTOF SCP
## Abstract
This protocol describes how to connect an IonOpticks column to a timsTOF SCP instrument using an Evosep One LC system. It includes steps for disconnecting the column, connecting the new column, and ensuring proper grounding and temperature control.
## Materials
### Equipment
- timsTOF SCP instrument
- IonOpticks column
- Evosep One LC system
- Column oven (Column Toaster)
- Sample line
- NanoViper adapter
- Pliers
### Reagents
- Acetonitrile (HPLC LC-MS grade)
## Procedure
*Estimated timing: 10 minutes*
1. **Switch the timsTOF SCP to standby mode.** In the timsControl software, click on the "on/off Instrument" button to change the software status from "Operating" to "Standby".
2. **Disconnect the old column.**
    a. Open the lid of the column oven (Column Toaster).
    c. Attach a NanoViper adapter to the sample line by gently pushing it onto the NanoViper connector.
    d. Hold the column fitting with pliers and unscrew the nano connector from the column fitting counter-clockwise.
3. **Connect the new IonOpticks column.**
    a. Attach a NanoViper adapter to the sample line.
    b. Screw the nano connector from the sample line into the column fitting clockwise.
    c. Remove the NanoViper adapter.
    d. Ensure that the column oven is as close as possible to the source.
    e. Ensure that the column is grounded. For shorter columns, use the screw on the source housing. For longer columns, use the screw on the column oven.
    f. Close the lid of the column oven.
    g. Ensure that the column oven is set to 50 degrees.
4. **Switch the timsTOF SCP to operate mode.** In the timsControl software, click on the "on/off Instrument" button to change the software status from "Standby" to "Operating".
5. **Check if the idle flow is on.** In the HyStar software, check if the idle flow is active. If not, right-click on the Evosep logo, select "idle flow", and then click "Run".
## Expected Results
After following this protocol, the IonOpticks column should be properly connected to the timsTOF SCP instrument and ready for use. The instrument should be in operate mode, the idle flow should be active, and the column oven should be at the correct temperature.
## Figures
- Extended Data Fig. 1: TimsControl settings
- Extended Data Fig. 2: Navigating in HyStar
- Supplementary Method Figure 1: UltraSource
- Supplementary Method Figure 4: Connection of a sample line of a LC system with a separation column and emitter.
## References
- Meier, F. et al. Online parallel accumulation-serial fragmentation (PASEF) with a novel trapped ion mobility mass spectrometer. Mol. Cell. Proteom. 17, 2534-2545 (2018).
- Skowronek, P. et al. Rapid and In-Depth Coverage of the (Phospho-)Proteome With Deep Libraries and Optimal Window Design for dia-PASEF. Mol. Cell. Proteomics 21, 100279 (2022).

**CONCLUSION:**

The provided video does not show all the steps described in the protocol. Specifically, the steps related to loading Evotips on the Evosep One are completely missing. Additionally, some steps in the column connection protocol, such as loosening the securing screw for grounding and placing the sample line on the bumper, are not explicitly shown.

The corrected protocol removes the missing steps and clarifies the remaining steps to ensure that they are directly observable in the video. This ensures that the protocol is accurate and reproducible, meeting the high standards of scientific rigor expected in the field of proteomics and mass spectrometry.
